#  *** DRAFT ***
##### Author: Omer Nivron
##### Date: 04/09/19

# Predict whether or not it will rain tomorrow in Australia

####  Data source: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package 

### Plan

1. Data cleanup
 - Formatting issues
 - Duplicates
 - Missing values
 
2. Exploration
 - visualizations
 
3. Feature Engineering
 - transform categorical features
 - 
 -

4. Modeling
    - Decision tree classifier
    - Random forest classifier

5. Evalution
    - misclassification error
    - feature importance

### Import packages

In [1]:
import pandas as pd
import numpy as np 
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree, ensemble
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

### Functions 

In [2]:
def extract_year_mon_day_cols_from_date_col(df):
    df['Year'], df['Month'], df['Day'] = df['Date'].dt.year, df['Date'].dt.month, df['Date'].dt.day
    return df

In [3]:
def remove_miss_col_with_bigger_than_share(df, share):
    df = df.loc[:, (df.isnull().sum(axis=0) / df.shape[0]) <= share]
    
    return df

In [4]:
def get_mode_col_w_missing_vals(df, col_name, groupby_cols = None):
    mode = df[groupby_cols + [col_name]][~df[col_name].isna()].groupby(groupby_cols) \
    .agg(lambda x: scipy.stats.mode(x)[0])[col_name].reset_index()
    # merge with df
    df = pd.merge(df, mode, on=groupby_cols, suffixes=('', '_mode'))
    return df

In [5]:
def get_median_col_w_missing_vals(df, col_name, groupby_cols = None):
    median = df[groupby_cols + [col_name]][~df[col_name] \
                                                               .isna()].groupby(groupby_cols) \
                                                            .median()[col_name].reset_index()
    # merge with df
    df = pd.merge(df, median, on=groupby_cols, suffixes=('', '_median'))
    return df

In [6]:
def wrapper_median_mode_df(df): 
    for column in list(df.columns[df.isna().sum() > 0]):
        if df[column].dtype == 'O':
            df = get_mode_col_w_missing_vals(df, column, ['Location','Year','Month'])
            df = if_na_get_mode_or_median(df, column, func = 'mode')
        else:
            df = get_median_col_w_missing_vals(df, column, ['Location','Year','Month'])
            df = if_na_get_mode_or_median(df, column, func = 'median')
    return df

In [7]:
def if_na_get_mode_or_median(df, col_name, func = 'mode'):
    df.loc[df[col_name].isna(), col_name] = df[df[col_name].isna()][col_name + '_' + func]
    df = df.drop(col_name + '_' + func, axis=1)
    return df

In [8]:
def if_no_rainToday_zero_rainfall(df):
    df['Rainfall'] = df[['Rainfall', 'RainToday']].apply(lambda x: x['Rainfall'] \
                                                                          if x['RainToday'] == 'Yes' else 0 , axis =1)
    
    return df

In [9]:
def transform_string_to_date(pd_series):
    pd_series = pd_series.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    return pd_series

In [10]:
def transform_yes_no_col_to_binary(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: 1 if x == 'Yes' else 0)
    return df

In [11]:
def transform_to_dummies(df, cols):
    dummies =  pd.get_dummies(df[cols])
    df = df.drop(cols, axis =1)
    df = pd.concat([df, dummies], axis=1)
    return df

In [12]:
def tr_te_split(df, traget_col, cond_col, cond):
    index_split = df[cond_col] > cond
    tr_x, tr_y, te_x, te_y = df.loc[index_split, df.columns != traget_col], \
    df.loc[index_split, traget_col], df.loc[~index_split, df.columns != traget_col], \
    df.loc[~index_split, traget_col] 
    
    return tr_x, tr_y, te_x, te_y

In [13]:
def calc_misclass_error(pred, target):
    error = np.where((pred == target) == False)[0].shape[0]/ target.shape[0]
    return error

### Read data 

In [14]:
australian_rain_df = pd.read_csv('/Users/onivron/Downloads/weatherAUS.csv')

#### Data statistics - get sense what values we have 

In [15]:
australian_rain_df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RISK_MM
count,141556.000000,141871.000000,140787.000000,81350.000000,74377.000000,132923.000000,140845.000000,139563.000000,140419.000000,138583.000000,128179.000000,128212.000000,88536.000000,85099.000000,141289.000000,139467.000000,142193.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235,2.360682
std,6.403283,7.117618,8.465173,4.188537,3.781525,13.588801,8.893337,8.803345,19.051293,20.797772,7.105476,7.036677,2.887016,2.720633,6.492838,6.937594,8.477969
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,0.000000
25%,7.600000,17.900000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.900000,1010.400000,1.000000,2.000000,12.300000,16.600000,0.000000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000,0.000000
75%,16.800000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1020.000000,7.000000,7.000000,21.600000,26.400000,0.800000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,371.000000


### Data cleaning

##### According to kaggle we should remove the column RISK_MM

In [16]:
australian_rain_df = australian_rain_df.drop('RISK_MM', axis=1)

#### Formatting - here we check the type each column has and change it if needed

In [85]:
pd.DataFrame(australian_rain_df.dtypes).transpose()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,Location_Townsville,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8


##### Let's check the type of the object columns

In [18]:
list(map(type, australian_rain_df.columns[australian_rain_df.dtypes == 'object']))

[str, str, str, str, str, str, str]

##### 1. Change Date column to date type and extract year, month and day columns



In [19]:
australian_rain_df['Date'] = transform_string_to_date(australian_rain_df['Date'])
australian_rain_df = extract_year_mon_day_cols_from_date_col(australian_rain_df)

##### 2. Change RainToday and RainTomorrow to 0/1 instead Yes/No
Preform this step only after treating the missing values (uncomment) 

In [27]:
australian_rain_df = transform_yes_no_col_to_binary(australian_rain_df, 'RainToday')
australian_rain_df = transform_yes_no_col_to_binary(australian_rain_df, 'RainTomorrow')

#### Duplicates - are there any repeating dates per location ? 

In [21]:
print("There are " + str(sum(australian_rain_df.duplicated(subset=['Date','Location'], keep=False))) + " duplicated dates here!")

There are 0 duplicated dates here!


##### Let's drop the Date column (we have the Year, Month and Day instead)

In [23]:
australian_rain_df = australian_rain_df.drop('Date', axis=1)

#### Missing values - imputation

In [81]:
 pd.DataFrame({'Missing_abs':[australian_rain_df.isna().sum()][0], \
               'Missing_share':[australian_rain_df.isna().sum() / australian_rain_df.shape[0]][0]}).transpose()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,Location_Townsville,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera
Missing_abs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Missing_share,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Let's remove columns with missing value share greater than 0.35

In [25]:
australian_rain_df = remove_miss_col_with_bigger_than_share(australian_rain_df, 0.35)

For the remaining columns with missing values if it's a categorical column we get the mode per location, year and month. If it is numeric column we get the median per location, year and month. After, We will enforce if there was no rain today then Rainfall equals 0.

In [26]:
australian_rain_df = wrapper_median_mode_df(australian_rain_df)
australian_rain_df = if_no_rainToday_zero_rainfall(australian_rain_df)

### Exploration

In [ ]:
australian_rain_df[['Date', 'Location']].groupby('Location').size()

##### Change location to see other distributions  

In [ ]:
loc = 'Adelaide'

##### For numeric variables let's plot boxplots per location and month

In [28]:
numeric_cols = australian_rain_df.columns[australian_rain_df.dtypes == 'float64']
cat_cols = australian_rain_df.columns[australian_rain_df.dtypes == 'object'][1:]

In [ ]:
cat_cols

In [ ]:
for num_var in numeric_cols:
    australian_rain_df[australian_rain_df['Location'] == loc].boxplot(column=num_var, by =['Month'])

##### For categorical variables let's plot histograms 

In [ ]:
#for cat_var in list(cat_cols):
plt.hist(australian_rain_df.loc[australian_rain_df['Location'] == loc, 'WindGustDir'])

##### Let's explore the correlations between variable pairs

In [ ]:
corr_plots = sns.PairGrid(australian_rain_df)
corr_plots.map(plt.scatter);

### Feature engineering

In [30]:
australian_rain_df = transform_to_dummies(australian_rain_df, list(cat_cols) + ['Location'])

### Modelling - decision tree and random forest classifiers

In [38]:
def model():
    feat_importance = {}
    results = {'Model': ['-'], 'Training misclassification': ['-'], 'Test misclassification': ['-'], 'Runtime': ['-']}
    results_pd = pd.DataFrame(data = results)
    models = ['Tree', 'Random forest']
    # split to training data and test data
    tr_x, tr_y, te_x, te_y = tr_te_split(australian_rain_df, 'RainTomorrow', 'Year', 2014)
    for kind in models:
        start = time.time()
        if kind == 'Tree':
            model = tree.DecisionTreeClassifier()
        else : 
            model = ensemble.RandomForestClassifier()
        
        model = model.fit(tr_x, tr_y)
        predict_tr = model.predict(tr_x)
        predict_te = model.predict(te_x)
        tr_err, te_err = calc_misclass_error(predict_tr, tr_y), calc_misclass_error(predict_te, te_y)
        feat_importance[kind] = model.feature_importances_
        end = time.time()
        runtime = round((end - start) / 60, 2)
        
        
        
        results_pd = results_pd.append(pd.DataFrame({'Model': [kind], 'Training misclassification': [str(round(tr_err, 2))], 
                                            'Test misclassification': [str(round(te_err, 2))], 'Runtime': [runtime]}))
        
    return feat_importance, results_pd

### Evaluation

In [39]:
feat_importance, results_pd = model()

In [42]:
results_pd.reset_index(drop=True)

,Model,Training misclassification,Test misclassification,Runtime
0,-,-,-,-
1,Tree,0.0,0.22,0.03
2,Random forest,0.01,0.16,0.03


In [76]:
df = australian_rain_df.drop('RainTomorrow', axis=1)

In [78]:
print('The top three features for the decision tree model are:' + ' ' + ', '.join(list(df.columns[feat_importance['Tree'].argsort()[:3]])))
print('The top three features for the Random Forest model are:' + ' ' + ', '.join(list(df.columns[feat_importance['Random forest'].argsort()[:3]])))

The top three features for the decision tree model are: Location_Launceston, Location_Katherine, Location_Darwin
The top three features for the Random Forest model are: Location_Launceston, Location_Katherine, Location_Uluru


### It seems like we have a major overfit here ! 